In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Host': 'books.toscrape.com',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0"
}

pagina_atual = [str("https://books.toscrape.com/catalogue/page-{}.html").format(i) for i in range(1, 51)] #pegar todos not a number e deletalos

product_link = []
product_name = []
product_price = []
product_stock = []

for link in pagina_atual:
    req = requests.get(link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    products = soup.find_all(class_='product_pod')

    for product in products:
        product_path = (product.find('a'))['href']
        product_link.append(f"https://books.toscrape.com/catalogue/{product_path} ")

        h3find = product.find('h3')
        title_text = h3find.find('a')['title']
        product_name.append(title_text)

        price = product.find(class_='price_color')
        price_text = price.text.strip().replace("Â", "")  # removendo o caractere indesejado

        product_price.append(price_text)  # Mantém o preço como string
        
        stock = product.find(class_='instock availability')
        disponibilidade = stock.text.strip().replace('\n', '')  # removendo as quebras de linhas
        product_stock.append(disponibilidade)

# Criando DataFrame
df = pd.DataFrame({"Link": product_link, "Nome": product_name, "Preço": product_price, "Disponibilidade": product_stock}, columns=["Link", "Nome", "Preço", "Disponibilidade"])

# CSV
df



,Link,Nome,Preço,Disponibilidade
0,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,£51.77,In stock
1,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,£53.74,In stock
2,https://books.toscrape.com/catalogue/soumissio...,Soumission,£50.10,In stock
3,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,£47.82,In stock
4,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,£54.23,In stock
...,...,...,...,...
995,https://books.toscrape.com/catalogue/alice-in-...,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock
996,https://books.toscrape.com/catalogue/ajin-demi...,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock
997,https://books.toscrape.com/catalogue/a-spys-de...,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock
998,https://books.toscrape.com/catalogue/1st-to-di...,1st to Die (Women's Murder Club #1),£53.98,In stock
